In [ ]:
from clearml import Model, Logger
import pandas as pd
from keras.models import load_model
from keras.utils import load_img, img_to_array
from keras.applications.efficientnet_v2 import preprocess_input
import numpy as np
from utility import preprocess_image, OptimizedRounder,mapping
from clearml import Task, TaskTypes
from PIL import Image
import os
import shap
from lime import lime_image
import matplotlib.pyplot as plt
from keras.utils import img_to_array,load_img
from skimage.segmentation import mark_boundaries

In [ ]:
task = Task.init(project_name='Diabetic_Retinopathy_Detection', 
                 task_name='Retinopathy Inference', 
                 task_type=TaskTypes.inference,
                 reuse_last_task_id=False
                 )

# Create an input model using the ClearML ID of a model already registered in the ClearML platform
model_path = Model(model_id="48a76b1277154398991d1d079db968ae").get_local_copy()
model = load_model(model_path)
print(model.summary())

In [ ]:
image_path = "inference_Data/001639a390f0.png"
x_val = np.empty((1, 224, 224, 3), dtype=np.uint8)
img = load_img(image_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = preprocess_input(img_array)
x_val[0,:,:,:] = img_array

In [ ]:
y_pred = model.predict(x_val)
print(f'Model Prediction: {y_pred}')

optR = OptimizedRounder()
coefficients = [0.49964604, 1.55479703, 2.4369177,  3.26701671] # Learnt while training the efficient net model.
print(f'Coefficients: {coefficients}')
prediction = optR.predict(y_pred, coefficients)
#print('Optirmized Prediction :{y_val_pred}')
print(prediction)

In [ ]:
masker = shap.maskers.Image('inpaint_telea',x_val[0].shape)
explainer = shap.Explainer(model, masker)

# here we use 500 evaluations of the underlying model to estimate the SHAP values
shap_values = explainer(x_val, max_evals=500)
shap.image_plot(shap_values)

In [ ]:
shap.image_plot(shap_values)
fig = plt.gcf()
fig.savefig('shap_explanation.png')

In [ ]:
ret = shap.image_plot(shap_values, x_val, show=True)
print(ret)
#plt.show()
# get current figure and save it
fig = plt.gcf()
fig.savefig('shap_image_plot.png')

In [ ]:
shap_values.shape

In [ ]:
import streamlit as st
shap.image_plot(shap_values, x_val)
st.pyplot()

In [ ]:
#LIME 
# Create a LIME explainer
explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(x_val[0], model.predict, top_labels=1, hide_color=0, num_samples=1000)
temp, mask = explanation.get_image_and_mask(0, positive_only=False, num_features=10, hide_rest=False)
fig, axe = plt.subplots(figsize=(7, 3.5))
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
axe.text(180, 190, 'Predicted:'+mapping.get(int(prediction[0].item())), bbox=dict(facecolor='red', alpha=0.0))
plt.show()

In [ ]:
model_path = Model(model_id="48a76b1277154398991d1d079db968ae").get_local_copy()
model = load_model(model_path)
image_path = 'inference_Data/001639a390f0.png'
x_val = np.empty((1, 224, 224, 3), dtype=np.uint8)
img = load_img(image_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = preprocess_input(img_array)
x_val[0,:,:,:] = img_array

masker = shap.maskers.Image('inpaint_telea',x_val[0].shape)
explainer = shap.Explainer(model, masker)

# here we use 500 evaluations of the underlying model to estimate the SHAP values
shap_values = explainer(x_val, max_evals=500)
print(shap_values.shape)

In [ ]:
shap.image_plot(shap_values)

In [ ]:
explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(x_val[0], model.predict, top_labels=1, hide_color=0, num_samples=1000)
temp, mask = explanation.get_image_and_mask(0, positive_only=False, num_features=10, hide_rest=False)
fig, axe = plt.subplots(figsize=(7, 3.5))
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
axe.text(180, 190, 'Predicted:'+mapping.get(int(prediction[0].item())), bbox=dict(facecolor='red', alpha=0.0))
plt.show()
